In [197]:
import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.tree.model.RandomForestModel
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import scala.collection.JavaConverters._ 
import scala.util.hashing.{ MurmurHash3 => MH3 }
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.row_number
import org.apache.spark.sql.types.StringType
import org.apache.spark.graphx._

import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.tree.model.RandomForestModel
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import scala.collection.JavaConverters._
import scala.util.hashing.{MurmurHash3=>MH3}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.row_number
import org.apache.spark.sql.types.StringType
import org.apache.spark.graphx._


In [22]:
val df0 = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("weatherAUS-clean-FINAL.csv")

df0: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 31 more fields]


In [23]:
df0.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- RainToday_Sc: integer (nullable = true)
 |--

In [25]:
df0.registerTempTable("ausRainData")

2020-06-06 12:44:28,741 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


In [26]:
val df1 = spark.sql("select * from ausRainData")

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 31 more fields]


In [27]:
df1.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- RainToday_Sc: integer (nullable = true)
 |--

In [199]:
val location = df1.select($"Location")
  .flatMap(x => Iterable(x(0).toString))

location: org.apache.spark.sql.Dataset[String] = [value: string]


In [202]:
val hash: RDD[(VertexId, String)] = location
  .rdd
  .distinct()
  .map(x => (MurmurHash3.stringHash(x), x))

hash: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, String)] = MapPartitionsRDD[475] at map at <console>:135


In [204]:
hash.toDF.show(df2.count().toInt)

+-----------+----------------+
|         _1|              _2|
+-----------+----------------+
| -403901479|    CoffsHarbour|
| -424784333|         Woomera|
| 1735662482|         Bendigo|
| 1885063312|         Mildura|
|-1931965857|        Watsonia|
|-1185911642|           Moree|
|-1501181004|         Penrith|
| 1612109533|      Wollongong|
|  737484909|        Richmond|
| 2126364083|    PerthAirport|
|  220478170|      SalmonGums|
|  461690079|   NorfolkIsland|
| 1374240089|       NorahHead|
|  160510013|        Portland|
| 1038357001|     Witchcliffe|
|-1394747672|          Cairns|
|-1407311822|      WaggaWagga|
|-2085394632|MelbourneAirport|
|-1067979720|       Newcastle|
| -942212088|      PearceRAAF|
|-1075208953|          Sydney|
|-2035869294|       GoldCoast|
|  903326363|        Brisbane|
|-2012030282|          Hobart|
| -213329084|      Launceston|
|  840634082|         Walpole|
|  288157406|    AliceSprings|
| -583384466|        Canberra|
| -365093340|        Dartmoor|
|  33701